# Authentication

- Node.js / Express - REST(REpresentational State Transfer) : 웹에 존재하는 모든 자원(이미지, 동영상, DB 자원)에 고유한 URI를 부여해 활용)
- Token Generation and Authencication
- CORS
- Mongoose ODM
- Angular 
  - Angular-CLI
  - Angular Router, HTTP Module
  - Angular2-JWT
  - Auth Guard 
  - Angular Flash Message Module
- Compile & Deploy

## Setup
- install NodeJS from https://nodejs.org
- Install MongoDB from https://mongodb.com  and Robo 3T from https://robomongo.org/
- Install and start multi-window CMD tool such as cmder from http://cmder.net
- Install express-generator using npm as : "npm install express-generator -g"
- Buils a basic web site structure as : 
  - express --view=pug meanauthapp
  - cd meanauthapp
  - npm install
- Start web server
  - nodemon ./bin/www
- Brouse the web as "http://localhost:3000"
- Edit meanauthapp/package.json and add the following pcakages in the dependencies & rerun "npm install"
  -  "bcryptjs": "*",
  -  "body-parser": "*",
  -  "cors": "*",
  -  "jsonwebtoken": "*",
  -  "mongoose": "*",
  -  "passport": "*",
  -  "passport-jwt": "*"
- Edit menauthapp/app.js as follows

In [ ]:
var createError = require('http-errors');
var express = require('express');
var path = require('path');
var cookieParser = require('cookie-parser');
var logger = require('morgan');
//========================================<
const bodyParser = require('body-parser');
const cors = require('cors');
const passport = require('passport');
const mongoose = require('mongoose');
const config = require('./config/database');

mongoose.Promise = require('bluebird');
mongoose.connect(config.database, { useNewUrlParser: true });
mongoose.connection.on('connected', () => {
  console.log('Connected to database ' + config.database);
});
mongoose.connection.on('error', (err) => {
  console.log('Database error ' + err);
});

//========================================>

var indexRouter = require('./routes/index');
var usersRouter = require('./routes/users');

var app = express();

// view engine setup
app.set('views', path.join(__dirname, 'views'));
app.set('view engine', 'pug');

//========================================<
// CORS Middleware
app.use(cors());
// Body Parser Middleware
app.use(bodyParser.json());
// Passport Middleware
app.use(passport.initialize());
app.use(passport.session());

require('./config/passport')(passport);
//========================================>

app.use(logger('dev'));
app.use(express.json());
app.use(express.urlencoded({ extended: false }));
app.use(cookieParser());
app.use(express.static(path.join(__dirname, 'public')));

app.use('/', indexRouter);
app.use('/users', usersRouter);

// catch 404 and forward to error handler
app.use(function(req, res, next) {
  next(createError(404));
});

// error handler
app.use(function(err, req, res, next) {
  // set locals, only providing error in development
  res.locals.message = err.message;
  res.locals.error = req.app.get('env') === 'development' ? err : {};

  // render the error page
  res.status(err.status || 500);
  res.render('error');
});

module.exports = app;


- Create meanauthapp/config/database.js and add the following lines

In [ ]:
module.exports = {
    database: 'mongodb://localhost:27017/meanauth',
    secret: 'yoursecret'
    }

- Create meanauthapp/config/passport.js and add the following lines

In [ ]:
const JwtStrategy = require('passport-jwt').Strategy;
const ExtractJwt = require('passport-jwt').ExtractJwt;
const User = require('../models/user');
const config = require('../config/database');

module.exports = function(passport) {
  let opts = {};
  opts.jwtFromRequest = ExtractJwt.fromAuthHeaderWithScheme('jwt');
  opts.secretOrKey = config.secret;
  passport.use(new JwtStrategy(opts, (jwt_payload, done) => {
    User.getUserById(jwt_payload.data._id, (err, user) => {
      if(err) {
        return done(err, false);
      }

      if(user) {
        return done(null, user);
      } else {
        return done(null, false);
      }
    });
  }));
}

- Edit menauthapp/routes/users.js and add the following lines

In [ ]:
const express = require('express');
const router = express.Router();
//=======================================<
const passport = require('passport');
const jwt = require('jsonwebtoken');
const config = require('../config/database');
const User = require('../models/user');

router.post('/register', function(req, res, next) {
  const newUser = new User({
    name: req.body.name,
    email: req.body.email,
    username: req.body.username,
    password: req.body.password
  });

  User.addUser(newUser, (err, user) => {
    if(err){
      res.json({success: false, msg:'Failed to register user', err: err})
    } else {
      res.json({success: true, msg:'User registered'})
    }
  });
});

//=======================================>

/* GET users listing. */
router.get('/', function(req, res, next) {
  res.send('respond with a resource');
});

//==================================================<

router.post('/authenticate', function(req, res, next) {
  const username = req.body.username;
  const password = req.body.password;
  
  User.getUserByUsername(username, (err, user) => {
    // if(err) throw err;
    if(!user) {
      return res.json({success:false, msg:'User not found'});
    }
    
    User.comparePassword(password, user.password, (err, isMatch) => {
      // if(err) throw err;
      if(isMatch) {
        const token = jwt.sign({data: user}, config.secret, {
          expiresIn: 604800  // 1 week in seconds
        });

        res.json({
          success: true,
          token: 'JWT '+token,
          user: {
            id: user._id,
            name: user.name,
            username: user.username,
            email: user.email
          }
        });
      } else {
        return res.json({success: false, msg:'Wrong password'});
      }
    })
  })
});

router.get('/profile', passport.authenticate('jwt', {session:false}), (req, res, next) => {
  res.json({user: req.user});
});

//==================================================>

module.exports = router;

- Create meanauthapp/models/users.js and add the following lines

In [ ]:
const mongoose = require('mongoose');
const bcrypt = require('bcryptjs');
const config = require('../config/database');

// User Schema
const UserSchema = mongoose.Schema({
  name: {
    type: String
  },
  email: {
    type: String,
    required: true
  },
  username: {
    type: String,
    required: true
  },
  password: {
    type: String,
    required: true
  }
});

const User = module.exports = mongoose.model('User', UserSchema);

module.exports.getUserById = function(id, callback){
  User.findById(id, callback);
}

module.exports.getUserByUsername = function(username, callback){
  const query = {username: username}
  User.findOne(query, callback);
}

module.exports.addUser = function(newUser, callback){
  bcrypt.genSalt(10, (err, salt) => {
    bcrypt.hash(newUser.password, salt, (err, hash) => {
      //if(err) throw err;
      newUser.password = hash;
      newUser.save(callback);
    });
  });
}

module.exports.comparePassword = function(candidatePassword, hash, callback){
  bcrypt.compare(candidatePassword, hash, (err, isMatch) => {
    //if(err) throw err;
    callback(null, isMatch);
  });
}

- Install postname (from https://www.getpostman.com/ ) and experiment as
  - request => POST
  - http://localhost/3000/users/register
  - headers => Content-Type = application/json
  - body => check raw
    - {
        "name":"John Doe",
        "email":"John@gmail.com",
        "username":"john",
        "password":"12345"
        }
  - click SEND
  - check the result fomr Postman and Robo 3T, respectively.

## Angular 

- remove all files in meanauthapp/public
- Install npm-cli as : "npm install -g angular-cli"
- execute the following commend in the folder meanauthapp : "ng new angular-src"
- edit engular.json file as : "outputPath": "../public"  
- Start abgular server as "ng serve"
- Brouse app as "localhost:4200"

### Components

- create navbar component in the folder meanauthapp/angular-src/app/components as "ng g c components/navbar"
- Edit ~ src/app/app.component.html as 

In [ ]:
<app-navbar></app-navbar>

- create login component in the folder meanauthapp/angular-src/app/components as "ng g c components/login"
- create home component in the folder meanauthapp/angular-src/app/components as "ng g c components/home"
- create dashboard component in the folder meanauthapp/angular-src/app/components as "ng g c components/dashboard"
- create profile component in the folder meanauthapp/angular-src/app/components as "ng g c components/profile"

### Apply "twitter bootstrap"
  - Edit ~ src/index.html and include the following :

In [ ]:
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css">
  <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.3/umd/popper.min.js"></script>
  <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/js/bootstrap.min.js"></script>

### Router
- Edit app.module.ts and insert lines as follows:

In [ ]:
import { BrowserModule } from '@angular/platform-browser';
import { NgModule } from '@angular/core';
import { FormsModule } from '@angular/forms';
import { HttpModule } from '@angular/http';
import { RouterModule, Routes } from '@angular/router';

import { AppComponent } from './app.component';
import { NavbarComponent } from './components/navbar/navbar.component';
import { LoginComponent } from './components/login/login.component';
import { RegisterComponent } from './components/register/register.component';
import { HomeComponent } from './components/home/home.component';
import { DashboardComponent } from './components/dashboard/dashboard.component';
import { ProfileComponent } from './components/profile/profile.component';

import { ValidateService } from './services/validate.service';
import { AuthService } from './services/auth.service';
import { NgFlashMessagesModule } from 'ng-flash-messages';
import { AuthGuard } from './guards/auth.guard';

const appRoutes: Routes = [
  {path:'', component: HomeComponent},
  {path:'register', component: RegisterComponent},
  {path:'login', component: LoginComponent},
  {path:'dashboard', component: DashboardComponent, canActivate:[AuthGuard]},
  {path:'profile', component: ProfileComponent, canActivate:[AuthGuard]}
]

@NgModule({
  declarations: [
    AppComponent,
    NavbarComponent,
    LoginComponent,
    RegisterComponent,
    HomeComponent,
    DashboardComponent,
    ProfileComponent
  ],
  imports: [
    BrowserModule,
    FormsModule,
    HttpModule,
    RouterModule.forRoot(appRoutes),
    NgFlashMessagesModule
  ],
  providers: [ValidateService, AuthService, AuthGuard],
  bootstrap: [AppComponent]
})
export class AppModule { }


- Edit ~ src/app/app.component.html as follows and test to brouse "localhost:4200/login" etc.

In [ ]:
<!--The content below is only a placeholder and can be replaced.-->
<app-navbar></app-navbar>
<div class="container">
  <ng-flash-message></ng-flash-message>
  <router-outlet></router-outlet>
</div>

- Edit ~  src/app/components/navbar.component.html as follows: (refer getbootstrap.com)

In [ ]:
<nav class="navbar navbar-expand-md navbar-dark bg-dark">
    <a class="navbar-brand" href="#">MEAN Auth App</a>
    <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarsExample04" aria-controls="navbarsExample04" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>

    <div class="collapse navbar-collapse" id="navbarsExample04">
      <ul class="navbar-nav mr-auto">
        <li class="nav-item active" [routerLinkActive]="['active']" [routerLinkActiveOptions]="{exact:true}" >
          <a class="nav-link" [routerLink]="['/']" >Home</a>
        </li>
      </ul>
      <ul class="navbar-nav navbar-right">
          <li *ngIf="authService.loggedIn()" class="nav-item" [routerLinkActive]="['active']" >
              <a class="nav-link" [routerLink]="['/dashboard']">Dashboard</a>
            </li>
            <li *ngIf="authService.loggedIn()" class="nav-item" [routerLinkActive]="['active']" >
                <a class="nav-link" [routerLink]="['/profile']">Profile</a>
              </li>
            <li *ngIf="!authService.loggedIn()" class="nav-item" [routerLinkActive]="['active']">
              <a class="nav-link"  [routerLink]="['/register']">Register</a>
            </li>
            <li *ngIf="!authService.loggedIn()" class="nav-item" [routerLinkActive]="['active']" >
              <a class="nav-link" [routerLink]="['/login']">Login</a>
            </li>
            <li class="nav-item">
              <a class="nav-link" (click)="onLogoutClick()" href="#">Logout</a>
            </li>
      </ul>
    </div>
</nav>

- Edit ~ src/app/components/home.component.html as follows: 

In [ ]:
<div class="jumbotron text-center">
  <h1>MEAN Authentication App</h1>
  <p class="lead">Welcome to our custom MEAN authentication application built from scratch.</p>
  <div>
    <a class="btn btn-primary" [routerLink]="['/register']">Register</a> 
    <a class="btn btn-secondary" [routerLink]="['/login']">Login</a>
  </div>
</div>

<div class="row">
  <div class="col-md-4">
    <h3>Extress Backend</h3>
    <p> A rock solid Node.js/Express server using Mongoose to organize models and query the database </p>
  </div>
  <div class="col-md-4">
    <h3>Angular-CLI</h3>
    <p> Angular0CLI to generate components, services and more. Local dev server and easy compilation </p>
  </div>
  <div class="col-md-4">
    <h3>JWT Tokens</h3>
    <p> Full featured authentication using JSON web tokens. Login and store user data </p>
  </div>
</div>

- Edit ~ src/app/components/register.component.ts as follows: 

In [ ]:
import { Component, OnInit } from '@angular/core';
import { ValidateService } from '../../services/validate.service';
import { AuthService } from '../../services/auth.service';
import { NgFlashMessageService } from 'ng-flash-messages';
import { Router } from '@angular/router';

@Component({
  selector: 'app-register',
  templateUrl: './register.component.html',
  styleUrls: ['./register.component.css']
})
export class RegisterComponent implements OnInit {
  name: String;
  username: String;
  email: String;
  password: String;

  constructor(
    private validateService: ValidateService, 
    private flashMessage: NgFlashMessageService, 
    private authService: AuthService,
    private router: Router
    ) { }

  ngOnInit() {
  }

  onRegisterSubmit() {
    const user = {
      name: this.name,
      username: this.username,
      email: this.email,
      password: this.password
    }

  // Required Fields
    if(!this.validateService.validateRegister(user)) {
      this.flashMessage.showFlashMessage({messages: ['Please fill in all fields'], type: 'danger', timeout:2000});
      return false;
    }

    // Validate Email
    if(!this.validateService.validateEmail(user.email)) {
      this.flashMessage.showFlashMessage({messages: ['Please use a valid email'], type: 'danger', timeout:2000});
      return false;
    }

    // Register User
    this.authService.registerUser(user).subscribe(data => {
      if(data.success) {
        this.flashMessage.showFlashMessage({
          messages: ['You are now registered and can login '], 
          type: 'success', 
          timeout:2000
        });
        this.router.navigate(['/']);
      } else {
        this.flashMessage.showFlashMessage(
          {messages: ['Something went wrong'], 
          type: 'danger', 
          timeout:3000
        });
        this.router.navigate(['/register']);
      }
    })
  }

}


- Edit ~ src/app/components/register.component.html as follows: 

In [ ]:
<h2 class="page-header"> Register</h2>
<form (submit)="onRegisterSubmit()">
  <div class="form-group">
    <label>Name</label>
    <input type="text" [(ngModel)]="name" name="name" class="form-control">
  </div>
  <div class="form-group">
    <label>Username</label>
    <input type="text" [(ngModel)]="username" name="username"  class="form-control">
  </div>
  <div class="form-group">
    <label>Email</label>
    <input type="text" [(ngModel)]="email" name="email" class="form-control">
  </div>
  <div class="form-group">
    <label>Password</label>
    <input type="password" [(ngModel)]="password" name="password" class="form-control">
  </div>
    <input type="submit" class="btn btn=primary" value="Submit">
</form>

### Services
- create validate service in the folder meanauthapp/angular-src/app/services as "ng g s services/validate"
- Edit ~ src/app/app.module.ts and insert lines as follows:

In [ ]:
import { ValidateService } from './services/validate.service';
...
  providers: [ValidateService],

- Edit ~ src/app/services/validate.service.ts as follows:

In [ ]:
import { Injectable } from '@angular/core';

@Injectable({
  providedIn: 'root'
})
export class ValidateService {

  constructor() { }

  validateRegister(user) {
    if(user.name == undefined || user.email == undefined || user.username == undefined || user.password == undefined) {
      return false;
    } else {
      return true;
    }
  }

  validateEmail(email) {
    var re = /^(([^<>()[\]\\.,;:\s@\"]+(\.[^<>()[\]\\.,;:\s@\"]+)*)|(\".+\"))@((\[[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\])|(([a-zA-Z\-0-9]+\.)+[a-zA-Z]{2,}))$/;
    return re.test(email);
  }
}


- install ng-flash-messages as "npm install ng-flash-messages --save"
- Edit ~ src/app/app.module.ts and insert lines as follows:

In [ ]:
import { NgFlashMessagesModule } from 'ng-flash-messages';
...
imports: [
    BrowserModule,
    FormsModule,
    HttpModule,
    RouterModule.forRoot(appRoutes),
    NgFlashMessagesModule
  ],

- Edit ~ src/app/app.component.html and insert lines as follows:

In [ ]:
<!--The content below is only a placeholder and can be replaced.-->
<app-navbar></app-navbar>
<div class="container">
  <ng-flash-message></ng-flash-message>  <!-- here -->
  <router-outlet></router-outlet>
</div>

- Edit ~ src/app/components/register.component.ts and insert lines as follows:

In [ ]:
...
import { NgFlashMessageService } from 'ng-flash-messages';
...
constructor(
    private validateService: ValidateService, 
    private flashMessage: NgFlashMessageService, 
    private authService: AuthService,
    private router: Router
    ) { }
...

  // Required Fields
    if(!this.validateService.validateRegister(user)) {
      this.flashMessage.showFlashMessage({messages: ['Please fill in all fields'], type: 'danger', timeout:2000});
      return false;
    }

    // Validate Email
    if(!this.validateService.validateEmail(user.email)) {
      this.flashMessage.showFlashMessage({messages: ['Please use a valid email'], type: 'danger', timeout:2000});
      return false;
    }

    // Register User
    this.authService.registerUser(user).subscribe(data => {
      if(data.success) {
        this.flashMessage.showFlashMessage({
          messages: ['You are now registered and can login '], 
          type: 'success', 
          timeout:2000
        });
        this.router.navigate(['/']);
      } else {
        this.flashMessage.showFlashMessage(
          {messages: ['Something went wrong'], 
          type: 'danger', 
          timeout:3000
        });
        this.router.navigate(['/register']);
      }
    })
  }

}

- create authentication service in the folder meanauthapp/angular-src/app/services as "ng g s services/auth"
- Edit ~ src/app/auth.service.ts and insert lines as follows:
- install angular2-jwt in Angular source folder as "npm install angular2-jwt --save"

In [ ]:
import { Injectable } from '@angular/core';
import { Http, Headers } from '@angular/http';
import { map } from 'rxjs/operators';
import { tokenNotExpired } from 'angular2-jwt';

@Injectable({
  providedIn: 'root'
})
export class AuthService {
  authToken: any;
  user: any;

  constructor(private http:Http) { }

  registerUser(user) {
    let headers = new Headers();
    headers.append('Content-Type', 'application/json');
    return this.http.post('http://localhost:3000/users/register', user, {headers: headers})
      .pipe(map(res => res.json()));
  }

  authenticateUser(user) {
    let headers = new Headers();
    headers.append('Content-Type', 'application/json');
    return this.http.post('http://localhost:3000/users/authenticate', user, {headers: headers})
      .pipe(map(res => res.json()));
  }

  getProfile() {
    let headers = new Headers();
    this.loadToken();
    headers.append('Authorization', this.authToken);
    headers.append('Content-Type', 'application/json');
    return this.http.get('http://localhost:3000/users/profile', {headers: headers})
      .pipe(map(res => res.json()));
  }

  storeUserData(token, user) {
    localStorage.setItem('id_token', token);
    localStorage.setItem('user', JSON.stringify(user));
    this.authToken = token;
    this.user = user;
  }

  loadToken() {
    const token = localStorage.getItem('id_token');
    this.authToken = token;
  }

   loggedIn() {
    return tokenNotExpired('id_token');
   }

  logout() {
    this.authToken = null;
    this.user = null;
    localStorage.clear();
  }
}
 

- Edit ~ src/app/components/login.component.html and insert lines as follows:

In [ ]:
<h2 class="page-header">Login</h2>
<form (submit)="onLoginSubmit()">
  <div class="form-group">
    <label>Username</label>
    <input type="text" class="form-control" [(ngModel)]='username' name="username">
  </div>
  <div class="form-group">
    <label>password</label>
    <input type="password" class="form-control" [(ngModel)]='password' name="password">
  </div>
  <input type="submit" class="btn btn-primary" value="Login">
</form>

- Edit ~ src/app/components/login.component.ts and insert lines as follows:

In [ ]:
import { Component, OnInit } from '@angular/core';
import { AuthService } from '../../services/auth.service';
import { NgFlashMessageService } from 'ng-flash-messages';
import { Router } from '@angular/router';

@Component({
  selector: 'app-login',
  templateUrl: './login.component.html',
  styleUrls: ['./login.component.css']
})
export class LoginComponent implements OnInit {
  username: String;
  password: String;

  constructor(
    private authService: AuthService,
    private router: Router,
    private flashMessage: NgFlashMessageService
    ) { }

  ngOnInit() {
  }

  onLoginSubmit() {
    // console.log(this.username);
    const user = {
      username: this.username,
      password: this.password
    }

    this.authService.authenticateUser(user).subscribe(data => {
      if(data.success) {
        this.authService.storeUserData(data.token, data.user);
        this.flashMessage.showFlashMessage({
          messages: [data.msg], 
          type: 'success', 
          timeout:3000
        });
        this.router.navigate(['dashboard']);
      } else {
        this.flashMessage.showFlashMessage({
            messages: [data.msg], 
            type: 'danger', 
            timeout:3000
          });
          this.router.navigate(['login']);
      }
    });
  }

}

- Edit ~  src/app/components/navbar.component.ts as follows: 

In [ ]:
import { Component, OnInit } from '@angular/core';
import { AuthService } from '../../services/auth.service';
import { NgFlashMessageService } from 'ng-flash-messages';
import { Router } from '@angular/router';

@Component({
  selector: 'app-navbar',
  templateUrl: './navbar.component.html',
  styleUrls: ['./navbar.component.css']
})
export class NavbarComponent implements OnInit {

  constructor(
    private authService: AuthService,
    private router: Router,
    private flashMessage: NgFlashMessageService
    ) { }

  ngOnInit() {
  }

  onLogoutClick() {
    this.authService.logout();
    this.flashMessage.showFlashMessage({
      messages: ['You are logged out'], 
      type: 'success', 
      timeout: 3000
    });
    this.router.navigate(['login']);
    return false;
  }

}


- Edit ~  src/app/components/profile.component.ts as follows: 

In [ ]:
import { Component, OnInit } from '@angular/core';
import { AuthService } from '../../services/auth.service';
import { Router } from '@angular/router';

@Component({
  selector: 'app-profile',
  templateUrl: './profile.component.html',
  styleUrls: ['./profile.component.css']
})
export class ProfileComponent implements OnInit {
  user: Object;

  constructor(
    private authService: AuthService,
    private router: Router
  ) { }

  ngOnInit() {
    this.authService.getProfile().subscribe(profile => {
      this.user = profile.user;
    }, 
    err => {
      console.log(err);
      return false;
    });
  }

}


- Edit ~  src/app/components/profile.component.html as follows: 

In [ ]:
<div *ngIf="user">
  <h2 class="page-header"> {{user.name}} </h2>
  <ul class="list-group">
    <li class="list-group-item">Username: {{user.username}}</li>  
    <li class="list-group-item">Email: {{user.email}}</li>  
  </ul>
</div>

- Create src/app/guards/auth.guard.ts  and insert the followings :

In [ ]:
import { Injectable } from '@angular/core';
import { Router, CanActivate } from '@angular/router';
import { AuthService } from '../services/auth.service';

@Injectable()
export class AuthGuard implements CanActivate {
  constructor(
    private authService:AuthService, 
    private router:Router
    ){
  }

   canActivate() {
    if(this.authService.loggedIn()) {
      return true;
    } else {
      this.router.navigate(['/login']);
      return false;
    }
  }
}

- insert the followings in ~ src/app/app.module.ts

In [ ]:
...
import { AuthGuard } from './guards/auth.guard';
...
const appRoutes: Routes = [
  {path:'', component: HomeComponent},
  {path:'register', component: RegisterComponent},
  {path:'login', component: LoginComponent},
  {path:'dashboard', component: DashboardComponent, canActivate:[AuthGuard]},
  {path:'profile', component: ProfileComponent, canActivate:[AuthGuard]}
]
...


- Edit ~ src/app/components/dashboard.component.html as follows:

In [ ]:
<h2 class="page-header"> Dashboard </h2>
<p> Welcome to Your dashboard</p>

## Build
- run as "ng build"
- check the folder meanauthapp/public 